In [ ]:
# Import packages
import pandas as pd
import numpy as np
import json
import re
import string

In [ ]:
# Select exam type
e_type = 'pde'

# Select txt file path
data = open(f"{e_type}_exam.txt")

dict_data = {'question': [],
             'answer': []}

# Switch determine if the last line is in question section or no
q_prev = False

for line in data.readlines():
  # Check if line is not blank
  if line != '\n':

    # Call the first word of the line is `head`
    head = line.split()[0]

    # Append new question if all of these meet:
    # - `q_prev` is False
    # - First character of `head` is numeric
    # - Last character of `head` is numeric or 'r'
    if (not q_prev) and head[0].isnumeric() and (head[-1].isnumeric() or head[-1] == 'r'):
      dict_data['question'].append(line)
      q_prev = True

    # Append new answer if all of these meet:
    # - `q_prev` is True
    # - `head` is 'A.' or '*A.'
    elif q_prev and (head == 'A.' or head == '*A.'):
      dict_data['answer'].append(line)
      q_prev = False

    # Add line to unfinished question
    elif q_prev:
      dict_data['question'][-1] += line

    # Add line to unfinished answer
    else:
      dict_data['answer'][-1] += line

# Convert `dict_data` to a DataFrame and make a copy of it
df0 = pd.DataFrame(dict_data)

print('Number of questions: ', len(df0))
print('Columns: ', df0.columns.values)

# df0['choices'].iloc[85]
# df0.iloc[193:200]

Number of questions:  106
Columns:  ['question' 'answer']


In [ ]:
# Mask looking for 'r' (dupplicated tag) after numbers
# Regex note:
# - `^`: start of string
# - `()`: group
# - `[1-9]`: any number from 1 to 9
# - `\d{0, 4}`: string contains from 0 to 4 digits
# - `r`: character 'r'
# - `\s`: space character
# - `-`: hyphen character
dup_mask = df0['question'].str.contains('^(?:[1-9]\d{0,4})r\s\-\s', regex=True)

print(f"Number of duplicated questions: {len(df0[dup_mask])}")

# Make a copy of `df0` after de-duplicate
df1 = df0[~dup_mask].reset_index(drop=True)

display(df1.iloc[193:196])

Number of duplicated questions: 0


,question,answer


In [ ]:
# Strip the numbers
df1['question'] = df1['question'].str.replace('^(?:[1-9]\d{0,4})\s\-\s', '', regex=True)

In [ ]:
# Make a copy of `df1`
df_a = df1.copy()
df_a

,question,answer
0,You need ads data to serve AI models and histo...,"A. Use Cloud Storage as a data warehouse, shel..."
1,You are collecting IoT sensor data from millio...,"A. Partition table data by create_date, locati..."
2,A live TV show asks viewers to cast votes usin...,A. Create a Memorystore instance with a high a...
3,A shipping company has live package-tracking d...,A. Re-create the table using data partitioning...
4,You are designing a data mesh on Google Cloud ...,A. 1. Create a single Dataplex virtual lake an...
...,...,...
101,You need to connect multiple applications with...,A. Add CIDR 0.0.0.0/0 network to Authorized Ne...
102,You are migrating a large number of files from...,"A. Set up Cloud Storage FUSE, and mount the Cl..."
103,You work for an airline and you need to store ...,A. Create a BigQuery table where each record h...
104,You need to look at BigQuery data from a speci...,A. Run a scheduled query to pull the necessary...


In [ ]:
df_a['answer'][30]

'A. Create a highly available Cloud SQL instance in region Create a highly available read replica in region B. Scale up read workloads by creating cascading read replicas in multiple regions. Backup the Cloud SQL instances to a multi-regional Cloud Storage bucket. Restore the Cloud SQL backup to a new instance in another region when Region A is down.\nB. Create a highly available Cloud SQL instance in region A. Scale up read workloads by creating read replicas in multiple regions. Promote one of the read replicas when region A is down.\n*C. Create a highly available Cloud SQL instance in region A. Create a highly available read replica in region B. Scale up read workloads by creating cascading read replicas in multiple regions. Promote the read replica in region B when region A is down.\nD. Create a highly available Cloud SQL instance in region A. Scale up read workloads by creating read replicas in the same region. Failover to the standby Cloud SQL instance when the primary instance f

In [ ]:
def double_clean_split(c_list):
  '''
  Split the `answer` data into multiple choices
  There are cases where text like is '...in region B. Scale up read...'
  can be unsuccessfully extracted if using only split by `X. `
  Therefore we use the double split method:
  - Pick the whole string after 'A. ' as `split_1`
  - Split the `split_1` using `\nX. '
  '''
  splitted_list = []

  # Convert single list with empty string into a list of empty list
  # so that `mono_correct_list` and `mono_incorrect_list` will not get error
  # while executing `for` loop
  if c_list == ['']:
    splitted_list = [[]]
  else:
    for item in c_list:
      # Split using the 'X. ' at the start of string
      # and ignore '' as index 0
      split_1 = re.split(r'^[A-Z]\.[ ]', item)[1]

      # Continue split using '\nX. '
      split_2 = re.split(r'\n[A-Z]\.[ ]', split_1)

      splitted_list.append(split_2)
  return splitted_list

def mono_correct_list(c_list):
  '''
  Pick the correct choice(s) out of the combine list
  '''
  mono_correct_list = []
  for small_list in c_list:
    # First item in each small list is the correct answer, append it to the
    # list of total correct answers
    mono_correct_list.append(small_list[0])
  return mono_correct_list

def mono_incorrect_list(c_list):
  '''
  Pick the incorrect choices out of the combine list
  '''
  mono_incorrect_list = []
  for small_list in c_list:
    # Index 0 is always correct answer
    # Index 1 and after are the incorrect answers if they exist
    if len(small_list) > 1:
      for item in small_list[1:]:
        mono_incorrect_list.append(item)
  return mono_incorrect_list


# Split the text at asterisk
df_a['c_split'] = df_a['answer'].str.split(r'\*(?=[A-Z]\.)')

# display(df_a.loc[5, 'c_split'])

# ======================================================================

# Pick incorrect answers as the first item before asterisk
df_a['incorrect_l'] = df_a['c_split'].str[:1]

# display(df_a.loc[5, 'incorrect_l'])

# Split the incorrect answers list to get separated answers
df_a['incorrect'] = df_a['incorrect_l'].apply(double_clean_split).str[0]

# display(df_a.loc[5, 'incorrect'])

# ======================================================================

# Pick answer(s) after asterisk only
df_a['correct_l'] = df_a['c_split'].str[1:]

# display(df_a.loc[5, 'correct_l'])

# Split the list of potential correct answers into nested lists
# (as multiple correct answers exist)
df_a['correct_multi'] = df_a['correct_l'].apply(double_clean_split)

# display(df_a.loc[30, 'correct_multi'])

# Pick the correct answers and add results into a mono list
df_a['correct'] = df_a['correct_multi'].apply(mono_correct_list)

# display(df_a.loc[30, 'correct'])

# Pick the incorrect answers and add results into a mono list
df_a['incorrect_mono'] = df_a['correct_multi'].apply(mono_incorrect_list)

# display(df_a.loc[30, 'incorrect_mono'])

# ======================================================================

# Append the 'incorrect_mono' into 'incorrect'
df_a['incorrect'] += df_a['incorrect_mono']

display(df_a.loc[5, 'incorrect'])

['Schedule a daily copy of the dataset to a backup region.',
 'Schedule a daily BigQuery snapshot of the table.',
 'Modify ETL job to load the data into both the current and another backup region.\n']

In [ ]:
def strip_newline(c_list):
  '''
  Remove new line character at the beginning and the end of string items in list
  '''
  return [item.rstrip('\n') for item in c_list]

# New order of columns
new_order = ['question', 'incorrect', 'correct', 'answer', 'c_split', 'incorrect_l',
       'correct_l', 'correct_multi', 'incorrect_mono']

# Make a copy of `df_a` with new order of columns
df_b = df_a[new_order].copy()

# Strip '\n' character in `incorrect` column
df_b['incorrect'] = df_b['incorrect'].apply(strip_newline)

display(df_b['incorrect'][5])

# # Strip '\n' character in `correct` column
df_b['correct'] = df_b['correct'].apply(strip_newline)

display(df_b['correct'][5])

['Schedule a daily copy of the dataset to a backup region.',
 'Schedule a daily BigQuery snapshot of the table.',
 'Modify ETL job to load the data into both the current and another backup region.']

['Schedule a daily export of the table to a Cloud Storage dual or multi-region bucket.']

In [ ]:
df_b

,question,incorrect,correct,answer,c_split,incorrect_l,correct_l,correct_multi,incorrect_mono
0,You need ads data to serve AI models and histo...,"[Use Cloud Storage as a data warehouse, shell ...",[Use Dataflow to identify longtail and outlier...,"A. Use Cloud Storage as a data warehouse, shel...","[A. Use Cloud Storage as a data warehouse, she...","[A. Use Cloud Storage as a data warehouse, she...",[B. Use Dataflow to identify longtail and outl...,[[Use Dataflow to identify longtail and outlie...,"[Use BigQuery to ingest, prepare, and then ana..."
1,You are collecting IoT sensor data from millio...,"[Partition table data by create_date, location...","[Partition table data by create_date, cluster ...","A. Partition table data by create_date, locati...","[A. Partition table data by create_date, locat...","[A. Partition table data by create_date, locat...","[B. Partition table data by create_date, clust...","[[Partition table data by create_date, cluster...","[Cluster table data by create_date, location_i..."
2,A live TV show asks viewers to cast votes usin...,[Create a Memorystore instance with a high ava...,[Write votes to a Pub/Sub topic and load into ...,A. Create a Memorystore instance with a high a...,[A. Create a Memorystore instance with a high ...,[A. Create a Memorystore instance with a high ...,[D. Write votes to a Pub/Sub topic and load in...,[[Write votes to a Pub/Sub topic and load into...,[]
3,A shipping company has live package-tracking d...,[Re-create the table using data partitioning o...,[Implement clustering in BigQuery on the packa...,A. Re-create the table using data partitioning...,[A. Re-create the table using data partitionin...,[A. Re-create the table using data partitionin...,[B. Implement clustering in BigQuery on the pa...,[[Implement clustering in BigQuery on the pack...,[Implement clustering in BigQuery on the inges...
4,You are designing a data mesh on Google Cloud ...,[1. Create a single Dataplex virtual lake and ...,[1. Create a Dataplex virtual lake for each da...,A. 1. Create a single Dataplex virtual lake an...,[A. 1. Create a single Dataplex virtual lake a...,[A. 1. Create a single Dataplex virtual lake a...,[D. 1. Create a Dataplex virtual lake for each...,[[1. Create a Dataplex virtual lake for each d...,[]
...,...,...,...,...,...,...,...,...,...
101,You need to connect multiple applications with...,[Add CIDR 0.0.0.0/0 network to Authorized Netw...,[Leave the Authorized Network empty. Use Cloud...,A. Add CIDR 0.0.0.0/0 network to Authorized Ne...,[A. Add CIDR 0.0.0.0/0 network to Authorized N...,[A. Add CIDR 0.0.0.0/0 network to Authorized N...,[C. Leave the Authorized Network empty. Use Cl...,[[Leave the Authorized Network empty. Use Clou...,[Add CIDR 0.0.0.0/0 network to Authorized Netw...
102,You are migrating a large number of files from...,"[Set up Cloud Storage FUSE, and mount the Clou...",[Create a new TSV file for the remaining files...,"A. Set up Cloud Storage FUSE, and mount the Cl...","[A. Set up Cloud Storage FUSE, and mount the C...","[A. Set up Cloud Storage FUSE, and mount the C...",[C. Create a new TSV file for the remaining fi...,[[Create a new TSV file for the remaining file...,[Update the file checksums in the TSV file fro...
103,You work for an airline and you need to store ...,[Create a BigQuery table where each record has...,[Create a BigQuery table partitioned by dateti...,A. Create a BigQuery table where each record h...,[A. Create a BigQuery table where each record ...,[A. Create a BigQuery table where each record ...,[B. Create a BigQuery table partitioned by dat...,[[Create a BigQuery table partitioned by datet...,[Create a BigQuery table partitioned by ingest...
104,You need to look at BigQuery data from a speci...,[Run a scheduled query to pull the necessary d...,[Create a materialized view based off of the q...,A. Run a scheduled query to pull the necessary...,[A. Run a scheduled query to pull the necessar...,[A. Run a scheduled query to pull the necessar...,

In [ ]:
# Convert nested list `incorrect` column to multiple columns
df_mi = pd.DataFrame(df_b['incorrect'].to_list())

df_mi

,0,1,2
0,"Use Cloud Storage as a data warehouse, shell s...","Use BigQuery to ingest, prepare, and then anal...",Use Cloud Composer to identify longtail and ou...
1,"Partition table data by create_date, location_...","Cluster table data by create_date, location_id...","Cluster table data by create_date, partition b..."
2,Create a Memorystore instance with a high avai...,Create a Cloud SQL for PostgreSQL database wit...,Write votes to a Pub/Sub topic and have Cloud ...
3,Re-create the table using data partitioning on...,Implement clustering in BigQuery on the ingest...,Tier older data onto Cloud Storage files and c...
4,1. Create a single Dataplex virtual lake and c...,1. Create a single Dataplex virtual lake and c...,1. Create a Dataplex virtual lake for each dat...
...,...,...,...
101,Add CIDR 0.0.0.0/0 network to Authorized Netwo...,Add all application networks to Authorized Net...,Add CIDR 0.0.0.0/0 network to Authorized Netwo...
102,"Set up Cloud Storage FUSE, and mount the Cloud...",Renew the TLS certificate of the HTTPS endpoin...,Update the file checksums in the TSV file from...
103,Create a BigQuery table where each record has ...,Create a BigQuery table partitioned by ingesti...,Create a BigQuery table with a datetime column...
104,Run a scheduled query to pull the necessary da...,Use a cached query to accelerate time to results.,Limit the query columns being pulled in the fi...


In [ ]:
# New name format as `incorrect_0`, `incorrect_1`,...
new_incorrect_cols = [f"incorrect_{i}" for i in df_mi.columns.values]

# Make a copy of `df_mi` and change its column names
df_mi_2 = df_mi.copy()
df_mi_2.columns = new_incorrect_cols

df_mi_2

,incorrect_0,incorrect_1,incorrect_2
0,"Use Cloud Storage as a data warehouse, shell s...","Use BigQuery to ingest, prepare, and then anal...",Use Cloud Composer to identify longtail and ou...
1,"Partition table data by create_date, location_...","Cluster table data by create_date, location_id...","Cluster table data by create_date, partition b..."
2,Create a Memorystore instance with a high avai...,Create a Cloud SQL for PostgreSQL database wit...,Write votes to a Pub/Sub topic and have Cloud ...
3,Re-create the table using data partitioning on...,Implement clustering in BigQuery on the ingest...,Tier older data onto Cloud Storage files and c...
4,1. Create a single Dataplex virtual lake and c...,1. Create a single Dataplex virtual lake and c...,1. Create a Dataplex virtual lake for each dat...
...,...,...,...
101,Add CIDR 0.0.0.0/0 network to Authorized Netwo...,Add all application networks to Authorized Net...,Add CIDR 0.0.0.0/0 network to Authorized Netwo...
102,"Set up Cloud Storage FUSE, and mount the Cloud...",Renew the TLS certificate of the HTTPS endpoin...,Update the file checksums in the TSV file from...
103,Create a BigQuery table where each record has ...,Create a BigQuery table partitioned by ingesti...,Create a BigQuery table with a datetime column...
104,Run a scheduled query to pull the necessary da...,Use a cached query to accelerate time to results.,Limit the query columns being pulled in the fi...


In [ ]:
# Convert nested list `correct` column to multiple columns
df_mc = pd.DataFrame(df_b['correct'].to_list())

# Check row where the second correct choice exists
df_mc.loc[df_mc[1] == df_mc[1], :]

,0,1
11,Ensure that the data analytics team members do...,Enforce access control in the policy tag taxon...
22,Use the Pub/Sub subscription retain-acked-mess...,Use Pub/Sub Seek with a timestamp.
25,Increase the maximum number of workers and red...,Increase the memory available to the Airflow w...


In [ ]:
# New name format as `correct_0`, `correct_1`,...
new_correct_cols = [f"correct_{i}" for i in df_mc.columns.values]

# Make a copy of `df_mc` and change its column names
df_mc_2 = df_mc.copy()
df_mc_2.columns = new_correct_cols

df_mc_2

,correct_0,correct_1
0,Use Dataflow to identify longtail and outlier ...,None
1,"Partition table data by create_date, cluster t...",None
2,Write votes to a Pub/Sub topic and load into b...,None
3,Implement clustering in BigQuery on the packag...,None
4,1. Create a Dataplex virtual lake for each dat...,None
...,...,...
101,Leave the Authorized Network empty. Use Cloud ...,None
102,Create a new TSV file for the remaining files ...,None
103,Create a BigQuery table partitioned by datetim...,None
104,Create a materialized view based off of the qu...,None


In [ ]:
# Concatenate `df_b['question']`, `df_mi_2` and `df_mc_2`
df_final = pd.concat([df_b[['question']], df_mi_2, df_mc_2], axis=1)

df_final

,question,incorrect_0,incorrect_1,incorrect_2,correct_0,correct_1
0,You need ads data to serve AI models and histo...,"Use Cloud Storage as a data warehouse, shell s...","Use BigQuery to ingest, prepare, and then anal...",Use Cloud Composer to identify longtail and ou...,Use Dataflow to identify longtail and outlier ...,None
1,You are collecting IoT sensor data from millio...,"Partition table data by create_date, location_...","Cluster table data by create_date, location_id...","Cluster table data by create_date, partition b...","Partition table data by create_date, cluster t...",None
2,A live TV show asks viewers to cast votes usin...,Create a Memorystore instance with a high avai...,Create a Cloud SQL for PostgreSQL database wit...,Write votes to a Pub/Sub topic and have Cloud ...,Write votes to a Pub/Sub topic and load into b...,None
3,A shipping company has live package-tracking d...,Re-create the table using data partitioning on...,Implement clustering in BigQuery on the ingest...,Tier older data onto Cloud Storage files and c...,Implement clustering in BigQuery on the packag...,None
4,You are designing a data mesh on Google Cloud ...,1. Create a single Dataplex virtual lake and c...,1. Create a single Dataplex virtual lake and c...,1. Create a Dataplex virtual lake for each dat...,1. Create a Dataplex virtual lake for each dat...,None
...,...,...,...,...,...,...
101,You need to connect multiple applications with...,Add CIDR 0.0.0.0/0 network to Authorized Netwo...,Add all application networks to Authorized Net...,Add CIDR 0.0.0.0/0 network to Authorized Netwo...,Leave the Authorized Network empty. Use Cloud ...,None
102,You are migrating a large number of files from...,"Set up Cloud Storage FUSE, and mount the Cloud...",Renew the TLS certificate of the HTTPS endpoin...,Update the file checksums in the TSV file from...,Create a new TSV file for the remaining files ...,None
103,You work for an airline and you need to store ...,Create a BigQuery table where each record has ...,Create a BigQuery table partitioned by ingesti...,Create a BigQuery table with a datetime column...,Create a BigQuery table partitioned by datetim...,None
104,You need to look at BigQuery data from a speci...,Run a scheduled query to pull the necessary da...,Use a cached query to accelerate time to results.,Limit the query columns being pulled in the fi...,Create a materialized view based off of the qu...,None


In [ ]:
# Export `df_final` to a CSV file
df_final.to_csv(f'{e_type}_bank.csv', index=False)

In [ ]:
incorrect_col = [col for col in df_final.columns if 'incorrect' in col]
df_final.loc[0][incorrect_col].tolist()

['Use Cloud Storage as a data warehouse, shell scripts for processing, and BigQuery to create views for desired datasets.',
 'Use BigQuery to ingest, prepare, and then analyze the data, and then run queries to create views.',
 'Use Cloud Composer to identify longtail and outlier data points, and then output a usable dataset to BigQuery.']